In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/Kaggle3"

In [ ]:
# %cd /content/drive/MyDrive/Kaggle3
!kaggle datasets download -d lexset/synthetic-asl-alphabet #

Dataset URL: https://www.kaggle.com/datasets/lexset/synthetic-asl-alphabet
License(s): copyright-authors
100% 6.57G/6.58G [03:08<00:00, 30.3MB/s]
100% 6.58G/6.58G [03:08<00:00, 37.6MB/s]


In [ ]:
!unzip \*.zip # unzipping the datasets

Streaming output truncated to the last 5000 lines.
  inflating: Train_Alphabet/U/6f3439aa-c9f3-4279-830c-e7b417425835.rgb_0000.png  
  inflating: Train_Alphabet/U/6fc70b21-eb34-42e6-8fa5-368a753a311e.rgb_0000.png  
  inflating: Train_Alphabet/U/700d1cf4-d43f-4d04-ba14-bbb6e9273c38.rgb_0000.png  
  inflating: Train_Alphabet/U/703451c9-380e-41a3-962e-0ef7290f04ec.rgb_0000.png  
  inflating: Train_Alphabet/U/7068eb2d-4d24-4a05-bddf-63fc4e296005.rgb_0000.png  
  inflating: Train_Alphabet/U/706cde46-580e-42d7-8752-2a751d4a2d1a.rgb_0000.png  
  inflating: Train_Alphabet/U/70a03f73-8d9b-48e1-b7d2-d1269fe44efc.rgb_0000.png  
  inflating: Train_Alphabet/U/70b8a34d-9529-4ed3-9570-cdf55ae9659e.rgb_0000.png  
  inflating: Train_Alphabet/U/70ddd4cc-3a15-4b37-bd9d-ea70752e74bb.rgb_0000.png  
  inflating: Train_Alphabet/U/71730f20-7e4b-4dcb-8987-a162076760a0.rgb_0000.png  
  inflating: Train_Alphabet/U/71f6eadc-d5ce-44b0-b98b-2174744a81d7.rgb_0000.png  
  inflating: Train_Alphabet/U/72159181-ebda-4b0

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
train_data_dir = '/content/Train_Alphabet/'
test_data_dir = '/content/Test_Alphabet/'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # Using 20% of data for validation

train_generator = train_datagen.flow_from_directory(
    train_data_dir,  # Replace with your dataset directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    test_data_dir,  # Replace with your dataset directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')


Found 19440 images belonging to 27 classes.
Found 540 images belonging to 27 classes.


In [ ]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

17225924/17225924 [==============================] - 0s 0us/step


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator)

Epoch 1/10
608/608 [==============================] - 490s 770ms/step - loss: 0.1800 - accuracy: 0.9498 - val_loss: 0.1200 - val_accuracy: 0.9667
Epoch 2/10
608/608 [==============================] - 438s 721ms/step - loss: 0.0581 - accuracy: 0.9849 - val_loss: 0.0241 - val_accuracy: 0.9926
Epoch 3/10
608/608 [==============================] - 441s 725ms/step - loss: 0.0316 - accuracy: 0.9908 - val_loss: 0.0947 - val_accuracy: 0.9667
Epoch 4/10
608/608 [==============================] - 433s 712ms/step - loss: 0.0327 - accuracy: 0.9910 - val_loss: 0.1331 - val_accuracy: 0.9667
Epoch 5/10
608/608 [==============================] - 429s 706ms/step - loss: 0.0292 - accuracy: 0.9920 - val_loss: 0.3262 - val_accuracy: 0.9204
Epoch 6/10
608/608 [==============================] - 435s 716ms/step - loss: 0.0232 - accuracy: 0.9937 - val_loss: 0.0244 - val_accuracy: 0.9963
Epoch 7/10
608/608 [==============================] - 438s 720ms/step - loss: 0.0263 - accuracy: 0.9925 - val_loss: 0.0066 -

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')


17/17 [==============================] - 11s 657ms/step - loss: 0.0319 - accuracy: 0.9926
Validation Loss: 0.03191307559609413
Validation Accuracy: 0.9925925731658936


In [ ]:
# Save the model
model.save('/content/mobilenet_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Get true labels and predicted labels
true_labels = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Predict on validation data
predictions = model.predict(validation_generator)
predicted_labels = np.argmax(predictions, axis=1)

# Classification report
print('Classification Report')
print(classification_report(true_labels, predicted_labels, target_names=class_labels))

# Confusion matrix
print('Confusion Matrix')
print(confusion_matrix(true_labels, predicted_labels))


17/17 [==============================] - 11s 608ms/step
Classification Report
              precision    recall  f1-score   support

           A       0.05      0.05      0.05        20
           B       0.00      0.00      0.00        20
       Blank       0.10      0.10      0.10        20
           C       0.00      0.00      0.00        20
           D       0.05      0.05      0.05        20
           E       0.00      0.00      0.00        20
           F       0.00      0.00      0.00        20
           G       0.00      0.00      0.00        20
           H       0.00      0.00      0.00        20
           I       0.10      0.10      0.10        20
           J       0.00      0.00      0.00        20
           K       0.05      0.05      0.05        20
           L       0.05      0.05      0.05        20
           M       0.05      0.05      0.05        20
           N       0.05      0.05      0.05        20
           O       0.15      0.15      0.15        20
   

SAVE MODEL

EFFECIENTNET

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # Using 20% of data for validation

train_generator = train_datagen.flow_from_directory(
    train_data_dir,  # Replace with your dataset directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    test_data_dir,  # Replace with your dataset directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False)  # Important: Disable shuffle for validation generator


Found 19440 images belonging to 27 classes.
Found 540 images belonging to 27 classes.


In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


16705208/16705208 [==============================] - 0s 0us/step


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator)


Epoch 1/10
608/608 [==============================] - 512s 771ms/step - loss: 0.2066 - accuracy: 0.9403 - val_loss: 15.3192 - val_accuracy: 0.0370
Epoch 2/10
608/608 [==============================] - 467s 768ms/step - loss: 0.0581 - accuracy: 0.9838 - val_loss: 773.0946 - val_accuracy: 0.0333
Epoch 3/10
608/608 [==============================] - 465s 764ms/step - loss: 0.0376 - accuracy: 0.9903 - val_loss: 106.5200 - val_accuracy: 0.0352
Epoch 4/10
608/608 [==============================] - 460s 756ms/step - loss: 0.0212 - accuracy: 0.9943 - val_loss: 29.8813 - val_accuracy: 0.0370
Epoch 5/10
608/608 [==============================] - 461s 758ms/step - loss: 0.0452 - accuracy: 0.9889 - val_loss: 22.1498 - val_accuracy: 0.0370
Epoch 6/10
608/608 [==============================] - 456s 750ms/step - loss: 0.0296 - accuracy: 0.9922 - val_loss: 11.7871 - val_accuracy: 0.0352
Epoch 7/10
608/608 [==============================] - 464s 763ms/step - loss: 0.0339 - accuracy: 0.9912 - val_loss: 

In [ ]:
model.save('efficientnet_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


RESNET

In [29]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix


In [30]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # Using 20% of data for validation

train_generator = train_datagen.flow_from_directory(
    train_data_dir,  # Replace with your dataset directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    test_data_dir,  # Replace with your dataset directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False)  # Important: Disable shuffle for validation generator


Found 19440 images belonging to 27 classes.
Found 540 images belonging to 27 classes.


In [31]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


94765736/94765736 [==============================] - 1s 0us/step


In [32]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator)


Epoch 1/10
608/608 [==============================] - 525s 801ms/step - loss: 0.3836 - accuracy: 0.8851 - val_loss: 24.1267 - val_accuracy: 0.0370
Epoch 2/10
608/608 [==============================] - 488s 803ms/step - loss: 0.0544 - accuracy: 0.9829 - val_loss: 0.1276 - val_accuracy: 0.9537
Epoch 3/10
608/608 [==============================] - 492s 809ms/step - loss: 0.0575 - accuracy: 0.9829 - val_loss: 0.0901 - val_accuracy: 0.9704
Epoch 4/10
608/608 [==============================] - 493s 810ms/step - loss: 0.0408 - accuracy: 0.9894 - val_loss: 0.5697 - val_accuracy: 0.9000
Epoch 5/10
608/608 [==============================] - 480s 789ms/step - loss: 0.0433 - accuracy: 0.9878 - val_loss: 0.0862 - val_accuracy: 0.9759
Epoch 6/10
608/608 [==============================] - 491s 807ms/step - loss: 0.0381 - accuracy: 0.9889 - val_loss: 0.0285 - val_accuracy: 0.9907
Epoch 7/10
608/608 [==============================] - 484s 796ms/step - loss: 0.0282 - accuracy: 0.9927 - val_loss: 0.0705 

In [ ]:
model.save('resnet_model.h5')


In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')
